# Data Collection - Data Sources Exploration

This is a rough notebook to get started in identifying, downloading data sources.
Also early version of functions needed will be coded in there before being moved to their relevant files.

The market data on the current shopping list is:

Sovereigns
-------------------
- US treasury note yields [wip]
- UK gilts [wip]
- DE bonds
- FR bonds

Inflation
-------------------
- EU CPI [wip]
- UK CPI [wip]
- US CPI  [wip]

Interest Rates
-------------------
- ECB [wip]
- BOE [wip]

FX Rates
-------------------
- EUR/GBP [wip]
- USD/EUR [wip]

Equity Indices
-------------------
- FTSE100 [wip]
- NASDAQ [wip]
- DOW [wip]
- S&P500 [wip]

Volatility
-------------------

Commodities 
-------------------
- Brent [wip]
- Grain [wip]
- Aluminium [wip]

GDP
-------------------

Production
-------------------

Confidence
-------------------


Known data sources
-------------------
Data provider https://www.alphavantage.co/ 

Data provider https://polygon.io/stocks

Data provider https://data.nasdaq.com/

NASDAQ Futures https://data.nasdaq.com/data/EUREX-eurex-futures-data

ECB Data Warehouse https://sdw.ecb.europa.eu/reports.do?node=1000003300


Packages
--------------------
quandl (https://data.nasdaq.com/tools/python)

## Development & Change Log
06/12/21 - Work on the automated merging of time series <br>
05/12/21 - Updating of the data_dl to specify the wanted time period in order to test the database management functions. Addition of parameter to select the futures contract date. Changed the months of the futures month catalogue to numerical values so it works better with dates as strings

In [5]:
import numpy as np
import pandas as pd
import quandl
import datacollection as daco
import datetime
import os
%load_ext autoreload
%autoreload 2

In [45]:
data = data_dl("NASDAQ EUREX Futures")
data

,Open
Date,
2021-11-24,25.25
2021-11-25,24.75
2021-11-26,26.00
2021-11-29,25.95
2021-11-30,27.20
2021-12-01,25.15
2021-12-02,26.60
2021-12-03,27.30


## Design of the automated merging functions of save_csv

In [46]:
nasdaq_eurex_dl = data

In [52]:
def save_csv(data, name, folder, mode="", debug=False):
    """
    Saves a dataframe to the requested folder
    data: dataframe to be saved
    name: name of the file to be created
    mode: this is to decide what to do if the file already exists
        overwrite: 
        merge: 
    location: folder where the file is to be created
    debug: triggers verbose for troubleshooting
    """
    
    # Saving to CSV!
    data.to_csv(path)
    
return

In [53]:
    
    name = "nasdaq_eurex_futures"
    folder = "docs"
    mode = ""
    debug = False
    
    # Builds path and file name
    path = os.getcwd()
    path = path.replace("\\src","\\" + folder + "\\" + name + ".csv")
    path = path.replace("\\","\\\\")
    if debug == True:
        print("path:", path)
    
    # Does the file alread exists?
    if os.path.exists(path) == False:
        # Yes - saving to CSV!
        data.to_csv(path)
    
    # If it exists then the mode parameter will determine what to do
    else:
        if mode == "overwrite":
        
        elif mode == "merge":
            # This will be based on a full outer join for now
            data_original = pd.read_csv(path)
            data = data_original.merge(data,how="outer")
        
        else:
            raise ValueError("The file already exists. Please use the mode parameter to determine what to do.")
        
        
    

True

In [18]:
# Code needed to reset the testing of the merging
folder = "data"
name = "futures_eurostoxx50_x21_a"
path1 = os.getcwd()
path1 = path1.replace("\\src","\\" + folder + "\\" + name + ".csv")
path1 = path1.replace("\\","\\\\")
path2 = path1.replace("futures_eurostoxx50_x21_a.csv","futures_eurostoxx50_x21_b.csv" )
data1 = pd.read_csv(path1)
data2 = pd.read_csv(path2)
data1, data2

(          Date   Open
 0   2021-11-01  19.15
 1   2021-11-02  19.10
 2   2021-11-03  18.60
 3   2021-11-04  17.50
 4   2021-11-05  17.55
 5   2021-11-08  18.55
 6   2021-11-09  18.75
 7   2021-11-10  18.85
 8   2021-11-11  18.45
 9   2021-11-12  18.05
 10  2021-11-15  17.20
 11  2021-11-16  17.00
 12  2021-11-17  16.75,
          Date   Open
 0  2021-11-24  25.25
 1  2021-11-25  24.75
 2  2021-11-26  26.00
 3  2021-11-29  25.95
 4  2021-11-30  27.20
 5  2021-12-01  25.15
 6  2021-12-02  26.60
 7  2021-12-03  27.30)

In [20]:
data1.info(), data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    13 non-null     object 
 1   Open    13 non-null     float64
dtypes: float64(1), object(1)
memory usage: 336.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    8 non-null      object 
 1   Open    8 non-null      float64
dtypes: float64(1), object(1)
memory usage: 256.0+ bytes


(None, None)

In [24]:
data1.merge(data2,how="outer")

,Date,Open
0,2021-11-01,19.15
1,2021-11-02,19.10
2,2021-11-03,18.60
3,2021-11-04,17.50
4,2021-11-05,17.55
5,2021-11-08,18.55
6,2021-11-09,18.75
7,2021-11-10,18.85
8,2021-11-11,18.45
9,2021-11-12,18.05


In [22]:
data1, data2

(          Date   Open
 0   2021-11-01  19.15
 1   2021-11-02  19.10
 2   2021-11-03  18.60
 3   2021-11-04  17.50
 4   2021-11-05  17.55
 5   2021-11-08  18.55
 6   2021-11-09  18.75
 7   2021-11-10  18.85
 8   2021-11-11  18.45
 9   2021-11-12  18.05
 10  2021-11-15  17.20
 11  2021-11-16  17.00
 12  2021-11-17  16.75,
          Date   Open
 0  2021-11-24  25.25
 1  2021-11-25  24.75
 2  2021-11-26  26.00
 3  2021-11-29  25.95
 4  2021-11-30  27.20
 5  2021-12-01  25.15
 6  2021-12-02  26.60
 7  2021-12-03  27.30)

In [96]:
data1.columns

Index(['Open'], dtype='object')

In [86]:
data_test.merge(test,how="inner",on="Open")

,Open


### Design of overwrite function

In [27]:
mode = 'overwrite'
folder = "data"
name = "futures_eurostoxx50_x21_c"
path = os.getcwd()
path1 = path.replace("\\src","\\" + folder + "\\" + name + ".csv")
path1 = path.replace("\\","\\\\")
    
if os.path.exists(path) == False:
    # Yes - saving to CSV!
    data.to_csv(path)

# If it exists then the mode parameter will determine what to do
else:
    if mode == "overwrite":
        data1.to_csv(path)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\maria\\Dropbox\\Romaria\\Studies\\market-dashboard\\src'

## Design of  the data catalogue functions

In [1]:
def datacat_admin(rebuild):
    """
    To administer the single data catalogue, which contains all the codes covered by the data retrieval functions.
    rebuild: True/False, to recreate the data catalogue based on the contents of the nasdaq_eurex
    """
    
    import os
    
    # Overall logic
    # A. (re) build the data catalogue
    #   1. Lists the files in the docs folder
    #   2. Opens each file and looks for the standardisation identifiers
    #   3. Extracts the standardisation identifiers
    #   4. Save the identifiers into the catalogue dataframe
    #   5. Restart fo the next file
    #   6. Saves the data catalogue in the doc folder
    # B. No idea yet
    
    # 1. Lists the files in the docs folder
    path = os.getcwd()
    path = path.replace("\\src","\\docs")
    files = os.listdir(path)
    
    # Removing '.ipynb_checkpoints'
    files = list(filter(lambda k: not'.ipynb_checkpoints' in k,files))
    
    # 2. Opens each file and looks for the standardisation identifiers
    content = n
    
    
    
    
    

### Now working on the standardisation of the documentation files

In [24]:
def docu_standardise(docu, meta_provider, meta_desc, meta_product):
    """
    Adds columns and values to data services documentation files so it can be consolidated into the single data catalogue
    """
    
    # 1. Receives a dataframe of the documentation file and the filename. The latter will be used to add the identifiers/metadata
    # 2. Uses the metada data to add the identifying columns
    # 3. Send back the documementation with the identifying columns
    
    

In [6]:
daco.docu_dl(source="")

Documentation available on the following:
['NASDAQ EUREX Futures']


In [14]:
docu = daco.docu_dl('NASDAQ EUREX Futures',save=True)

In [17]:
df = docu
file_name = "nasdaq_eurex_futures.csv"
meta_provider = "nasdaq"
meta_desc = "eurex"
meta_product = "futures"

# Adding the new columns with the details
docu["meta_provider"] = meta_provider
docu["meta_desc"] = meta_desc
docu["meta_product"] = meta_product




,Symbol,Exchange,Name,Months,Quandl Code,meta_provider,meta_desc,meta_subdesc
0,CONF,EUREX,CONF,HMUZ,EUREX/CONF,nasdaq,eurex,futures
1,F2CR,EUREX,CER Emissions Mid December,Z,EUREX/F2CR,nasdaq,eurex,futures
2,F2MX,EUREX,MDAX,HMUZ,EUREX/F2MX,nasdaq,eurex,futures
3,FBTP,EUREX,Long-Term Euro-BTP,HMUZ,EUREX/FBTP,nasdaq,eurex,futures
4,FBTS,EUREX,Short-Term Euro-BTP,HMUZ,EUREX/FBTS,nasdaq,eurex,futures
...,...,...,...,...,...,...,...,...
66,FSMP,EUREX,Skimmed Milk Powder (EUREX),FGHJKMNQUVXZ,EUREX/FSMP,nasdaq,eurex,futures
67,FSTX,EUREX,STOXX Europe 50 Index,HMUZ,EUREX/FSTX,nasdaq,eurex,futures
68,FTDX,EUREX,DivDAX,HMUZ,EUREX/FTDX,nasdaq,eurex,futures
69,FVS,EUREX,EURO STOXX 50 Volatility (VSTOXX),HMUZ,EUREX/FVS,nasdaq,eurex,futures


## CHANGES TO dl_data

In [75]:
def data_dl (start_date, end_date, dataref="", contract_date=""):
    """
    Universal data download function that handles the different API calls based
    on what is being requested. Will ask for your identifiers where required.
    parameters:
        dataref: to provide the reference of the data to extract. The reference can be found in the datacatalogue.
                leave blank for dictionary of available sources
        start_date in YYYY-MM-DD format
        end_date in YYYY-MM-DD format
        contract_date in YYYY-MM, if left blank the end_date will be used to derive the futures contract date
    
    Returns a time series.
    """
    
    # Overall logic
    # 1. Provide datareference and dates
    #   a. If you don't know them you can retrieve a full list of what is available in the function
    #   b. But for this list you need a consolidated list of catalogues!
    # 2. The function retrieves the codes to get the data so you do not have to remember
    #   a. Retrieves the codes from files for the service requested
    #   b. If not available on the file then goes and download them from the support location
    #   c. Builds the request
    # 3. Sends the request to the API
    # 4. If the time period isn't available then the next best?
    # OUTPUT: data file downloaded

    # To do - use the data catalogue function here
    available = [
        "EUREX Futures"
    ]

    # Loads up all the available API to only have one call to the file
    api_dictionary = daco.apidict("read")

    # Shows data sources for which API calls are set up
    # Otherwise proceeds with the downloads
    if dataref == "":
        return available

    elif dataref == "EUREX Futures":
        import quandl

        # Sets up the API based on the dictionary
        quandl.ApiConfig.api_key = api_dictionary["NASDAQ"]

        # Futures delivery month codes
        futures_delivery_month = {
                "01": "F",
                "02": "G",
                "03": "H",
                "04": "J",
                "05": "K",
                "06": "M",
                "07": "N",
                "08": "Q",
                "09": "U",
                "10": "V",
                "11": "X",
                "12": "Z"
        }

        # Building the query
        # TO DO - retrieve the code dynamically via the catalogue
        query = {
            "code" : "EUREX/FVS",
            "month" : "",
            "year" : ""
        }

        # Translating the date to the futures format
        # The end date is used to work out the contract date if not provided
        if contract_date == "":
            query['year'] = end_date[0:4]
            query['month'] = futures_delivery_month[end_date[5:7]]
            query = query["code"] + query["month"] + query["year"]
            
        # Otherwise we use the contract date provided
        else: 
            query['year'] = contract_date[0:4]
            query['month'] = futures_delivery_month[contract_date[5:7]]
            query = query["code"] + query["month"] + query["year"]

        data = quandl.get(query, column_index='1',start_date=start_date, end_date=end_date)
        
    return data

In [76]:
test = data_dl("2021-11-01","2021-11-24",dataref="EUREX Futures")
test

,Open
Date,
2021-11-01,19.15
2021-11-02,19.10
2021-11-03,18.60
2021-11-04,17.50
2021-11-05,17.55
2021-11-08,18.55
2021-11-09,18.75
2021-11-10,18.85
2021-11-11,18.45


In [65]:
end_date = "2021-11-24"

In [73]:
futures_delivery_month = {
        "01": "F",
        "02": "G",
        "03": "H",
        "04": "J",
        "05": "K",
        "06": "M",
        "07": "N",
        "08": "Q",
        "09": "U",
        "10": "V",
        "11": "X",
        "12": "Z"
}
query = {
    "code" : "EUREX/FVS",
    "month" : "",
    "year" : ""
}
query['year'] = end_date[0:4]
query['month'] = futures_delivery_month[end_date[5:7]]
query = query["code"] + query["month"] + query["year"]
query

'EUREX/FVSX2021'

'11'